# Poisson Processes via Kinetic Monte Carlo

This notebook builds homogeneous and inhomogeneous Poisson processes
directly from the kinetic Monte Carlo primitives:
- `next_time` for exponential waiting times
- `next_event` for event selection
- `step!` for updating algorithm time and steps

In [1]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.develop(path=joinpath(@__DIR__, ".."))
Pkg.instantiate()

using Random
using StatsBase
using BenchmarkTools
using MonteCarloX

  Activating project at `~/.julia/dev/MonteCarloX/notebooks`
   Resolving package versions...
     Project No packages added to or removed from `~/.julia/dev/MonteCarloX/notebooks/Project.toml`
    Manifest No packages added to or removed from `~/.julia/dev/MonteCarloX/notebooks/Manifest.toml`
Precompiling packages...
   2534.1 ms  ✓ MonteCarloX
   1362.2 ms  ✓ SpinSystems
  2 dependencies successfully precompiled in 14 seconds. 206 already precompiled.


## Homogeneous Poisson process (single channel)

A constant rate can be modeled as a single-channel KMC process.
The inter-arrival times are exponentially distributed with mean $1/\lambda$.

In [2]:
rng = MersenneTwister(7)
alg = Gillespie(rng)

rate = 1.2
T = 10.0
arrival_times = Float64[]

while alg.time < T
    t_new, event = step!(alg, [rate])
    push!(arrival_times, t_new)
end

inter_arrival = diff([0.0; arrival_times])
println("Events: $(alg.steps)")
println("Mean inter-arrival: $(round(mean(inter_arrival), digits=3))")
println("Expected mean:      $(round(1 / rate, digits=3))")

@btime begin
    rng = MersenneTwister(7)
    alg = Gillespie(rng)
    rates = [$rate]
    arrival_times = Float64[]
    while alg.time < $T
        t_new, event = step!(alg, rates)
        push!(arrival_times, t_new)
    end
    arrival_times
end

Events: 13
Mean inter-arrival: 0.825
Expected mean:      0.833
  10.084 μs (29 allocations: 20.83 KiB)


13-element Vector{Float64}:
  0.824277486903756
  1.6551023627317205
  2.49722809142005
  2.921287732163237
  4.151929369101444
  4.816679129003675
  4.844921006719506
  5.6466421842874635
  5.729715924137639
  6.194627373444033
  8.351449806827983
  9.536112642527673
 10.723447667965385

In [3]:
rate = 1.2
T = 10.0

@btime begin
    rng = MersenneTwister(7)
    t = 0.0
    arrival_times = Float64[]
    while t < $T
        t += randexp(rng) / $rate
        push!(arrival_times, t)
    end
    arrival_times
end

  9.791 μs (26 allocations: 20.73 KiB)


13-element Vector{Float64}:
  0.824277486903756
  1.6551023627317205
  2.49722809142005
  2.921287732163237
  4.151929369101444
  4.816679129003675
  4.844921006719506
  5.6466421842874635
  5.729715924137639
  6.194627373444033
  8.351449806827983
  9.536112642527673
 10.723447667965385

## Inhomogeneous Poisson via thinning

We use a time-dependent rate vector and sample waiting times
from a dominating homogeneous Poisson process.
Accepted events are chosen using `next_event`.

In [4]:
rng = MersenneTwister(21)
alg = Gillespie(rng)

rate_vec(t) = [0.6 + 0.3 * sin(t), 0.7 + 0.2 * cos(t)]
generation_rate = [1.0, 1.0]

function thin_step!(alg, rate_vec, generation_rate)
    t0 = alg.time
    total_generation = sum(generation_rate)
    dt = next_time(alg.rng, τ -> sum(rate_vec(t0 + τ)), total_generation)
    t_new = t0 + dt
    rates_now = rate_vec(t_new)
    event = next_event(alg.rng, rates_now)
    alg.time = t_new
    alg.steps += 1
    return t_new, event
end

T = 20.0
counts = zeros(Int, 2)
times = Float64[]

while alg.time < T
    t_new, event = thin_step!(alg, rate_vec, generation_rate)
    counts[event] += 1
    push!(times, t_new)
end

println("Events: $(alg.steps)")
println("Channel counts: $(counts)")
println("Final time: $(round(alg.time, digits=3))")

Events: 25
Channel counts: [14, 11]
Final time: 20.015
